In [1]:
import os
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
# from tensorflow.keras.layers.experimental.preprocessing import *
from tensorflow.keras.optimizers import SGD, Adam
from keras.losses import BinaryCrossentropy
from keras.regularizers import l2

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
DATA_DIR = 'data'
HDF5_FILE = 'data_tf.hdf5'

In [4]:
BATCH_SIZE = 64
IMG_SHAPE = (96, 96, 4)

In [5]:
data_file = os.path.join(DATA_DIR, HDF5_FILE)
ds_x = tfio.IODataset.from_hdf5(data_file, '/x_train')
ds_y = tfio.IODataset.from_hdf5(data_file, '/y_train')

In [6]:
# convert to float and scale to [0, 1)
ds_x = ds_x.map(lambda x: tf.image.convert_image_dtype(x, tf.float32))
# ds_x = ds_images.map(lambda x: tf.image.per_image_standardization(x))
ds_xc = ds_x.map(lambda x: tf.add(x, tf.random.normal(tf.shape(x), 0, 0.2)))

In [7]:
ds_xy = tf.data.Dataset.zip((ds_x, ds_y))
ds_xy = ds_xy.shuffle(BATCH_SIZE*2)

ds_xcy = tf.data.Dataset.zip((ds_xc, ds_y))
ds_xcy = ds_xcy.shuffle(BATCH_SIZE*2)

In [8]:
ds_len = ds_xy.reduce(0, lambda x, _: x + 1).numpy()
train_len = int(0.9 * ds_len)

ds_train = ds_xy.take(train_len)
ds_trainc = ds_xcy.take(train_len)
ds_test = ds_xy.skip(train_len)

ds_len, train_len

(220025, 198022)

In [9]:
data_aug = Sequential([
    RandomFlip("horizontal_and_vertical"),
    # RandomZoom(0.2, 0.2)
])

In [10]:
model = Sequential()

model.add(data_aug)

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4))) #, input_shape=(X_train.shape[1:])))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D (pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D (pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# model.summary()

In [11]:
ds_trainc = ds_trainc.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-3),
              metrics=['accuracy'])

history = model.fit(ds_trainc,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
3095/3095 - 156s - loss: 0.4885 - accuracy: 0.8193 - val_loss: 0.8914 - val_accuracy: 0.7249
Epoch 2/8
3095/3095 - 153s - loss: 0.3730 - accuracy: 0.8601 - val_loss: 1.0383 - val_accuracy: 0.7532
Epoch 3/8
3095/3095 - 153s - loss: 0.3268 - accuracy: 0.8819 - val_loss: 0.5585 - val_accuracy: 0.8506
Epoch 4/8
3095/3095 - 153s - loss: 0.3050 - accuracy: 0.8915 - val_loss: 0.6478 - val_accuracy: 0.8140
Epoch 5/8
3095/3095 - 152s - loss: 0.2943 - accuracy: 0.8973 - val_loss: 0.7751 - val_accuracy: 0.7968
Epoch 6/8
3095/3095 - 152s - loss: 0.2866 - accuracy: 0.9004 - val_loss: 0.7867 - val_accuracy: 0.8189
Epoch 7/8
3095/3095 - 152s - loss: 0.2844 - accuracy: 0.9021 - val_loss: 1.1115 - val_accuracy: 0.7607
Epoch 8/8
3095/3095 - 153s - loss: 0.2811 - accuracy: 0.9047 - val_loss: 1.0483 - val_accuracy: 0.7693


In [12]:
ds_train = ds_train.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

history = model.fit(ds_train,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
3095/3095 - 152s - loss: 0.2348 - accuracy: 0.9262 - val_loss: 0.2552 - val_accuracy: 0.9185
Epoch 2/8
3095/3095 - 153s - loss: 0.2259 - accuracy: 0.9305 - val_loss: 0.3820 - val_accuracy: 0.8724
Epoch 3/8
3095/3095 - 159s - loss: 0.2215 - accuracy: 0.9322 - val_loss: 0.2866 - val_accuracy: 0.9113
Epoch 4/8
3095/3095 - 154s - loss: 0.2178 - accuracy: 0.9338 - val_loss: 0.3010 - val_accuracy: 0.9052
Epoch 5/8
3095/3095 - 153s - loss: 0.2145 - accuracy: 0.9351 - val_loss: 0.2432 - val_accuracy: 0.9264
Epoch 6/8
3095/3095 - 153s - loss: 0.2131 - accuracy: 0.9354 - val_loss: 0.2317 - val_accuracy: 0.9340
Epoch 7/8
3095/3095 - 154s - loss: 0.2122 - accuracy: 0.9367 - val_loss: 0.2377 - val_accuracy: 0.9254
Epoch 8/8
3095/3095 - 154s - loss: 0.2104 - accuracy: 0.9371 - val_loss: 0.5315 - val_accuracy: 0.8307


In [13]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(128).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=2e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
1548/1548 - 164s - loss: 0.1719 - accuracy: 0.9526 - val_loss: 0.1550 - val_accuracy: 0.9588
Epoch 2/8
1548/1548 - 151s - loss: 0.1621 - accuracy: 0.9550 - val_loss: 0.2063 - val_accuracy: 0.9386
Epoch 3/8
1548/1548 - 162s - loss: 0.1576 - accuracy: 0.9560 - val_loss: 0.1492 - val_accuracy: 0.9601
Epoch 4/8
1548/1548 - 151s - loss: 0.1538 - accuracy: 0.9568 - val_loss: 0.1470 - val_accuracy: 0.9599
Epoch 5/8
1548/1548 - 151s - loss: 0.1507 - accuracy: 0.9577 - val_loss: 0.1436 - val_accuracy: 0.9614
Epoch 6/8
1548/1548 - 162s - loss: 0.1483 - accuracy: 0.9581 - val_loss: 0.1613 - val_accuracy: 0.9529
Epoch 7/8
1548/1548 - 154s - loss: 0.1462 - accuracy: 0.9584 - val_loss: 0.1396 - val_accuracy: 0.9621
Epoch 8/8
1548/1548 - 152s - loss: 0.1447 - accuracy: 0.9585 - val_loss: 0.1416 - val_accuracy: 0.9605


In [14]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=8,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/8
774/774 - 156s - loss: 0.1333 - accuracy: 0.9628 - val_loss: 0.1271 - val_accuracy: 0.9652
Epoch 2/8
774/774 - 155s - loss: 0.1316 - accuracy: 0.9633 - val_loss: 0.1254 - val_accuracy: 0.9668
Epoch 3/8
774/774 - 155s - loss: 0.1303 - accuracy: 0.9637 - val_loss: 0.1259 - val_accuracy: 0.9664
Epoch 4/8
774/774 - 150s - loss: 0.1301 - accuracy: 0.9641 - val_loss: 0.1268 - val_accuracy: 0.9663
Epoch 5/8
774/774 - 150s - loss: 0.1288 - accuracy: 0.9642 - val_loss: 0.1231 - val_accuracy: 0.9675
Epoch 6/8
774/774 - 150s - loss: 0.1279 - accuracy: 0.9647 - val_loss: 0.1241 - val_accuracy: 0.9670
Epoch 7/8
774/774 - 150s - loss: 0.1270 - accuracy: 0.9648 - val_loss: 0.1241 - val_accuracy: 0.9662
Epoch 8/8
774/774 - 150s - loss: 0.1266 - accuracy: 0.9651 - val_loss: 0.1206 - val_accuracy: 0.9671


In [15]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(64).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(64).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=2,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/2
3095/3095 - 151s - loss: 0.1432 - accuracy: 0.9580 - val_loss: 0.1351 - val_accuracy: 0.9614
Epoch 2/2
3095/3095 - 162s - loss: 0.1429 - accuracy: 0.9581 - val_loss: 0.1363 - val_accuracy: 0.9624


In [16]:
history = model.fit(ds_trainc,
                    epochs=2,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/2
3095/3095 - 160s - loss: 0.2356 - accuracy: 0.9167 - val_loss: 1.0234 - val_accuracy: 0.8088
Epoch 2/2
3095/3095 - 160s - loss: 0.2176 - accuracy: 0.9241 - val_loss: 0.8157 - val_accuracy: 0.8287


In [17]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=5,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/5
774/774 - 161s - loss: 0.1418 - accuracy: 0.9580 - val_loss: 0.1621 - val_accuracy: 0.9484
Epoch 2/5
774/774 - 151s - loss: 0.1326 - accuracy: 0.9618 - val_loss: 0.1446 - val_accuracy: 0.9570
Epoch 3/5
774/774 - 164s - loss: 0.1284 - accuracy: 0.9637 - val_loss: 0.1305 - val_accuracy: 0.9625
Epoch 4/5
774/774 - 167s - loss: 0.1261 - accuracy: 0.9640 - val_loss: 0.1276 - val_accuracy: 0.9645
Epoch 5/5
774/774 - 166s - loss: 0.1243 - accuracy: 0.9650 - val_loss: 0.1197 - val_accuracy: 0.9670


In [18]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(512).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(512).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=2,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/2
387/387 - 170s - loss: 0.1302 - accuracy: 0.9630 - val_loss: 0.1458 - val_accuracy: 0.9595
Epoch 2/2
387/387 - 162s - loss: 0.1286 - accuracy: 0.9630 - val_loss: 0.1497 - val_accuracy: 0.9562


In [19]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
387/387 - 165s - loss: 0.1205 - accuracy: 0.9666 - val_loss: 0.1249 - val_accuracy: 0.9650
Epoch 2/10
387/387 - 162s - loss: 0.1175 - accuracy: 0.9674 - val_loss: 0.1180 - val_accuracy: 0.9672
Epoch 3/10
387/387 - 163s - loss: 0.1174 - accuracy: 0.9673 - val_loss: 0.1259 - val_accuracy: 0.9656
Epoch 4/10
387/387 - 162s - loss: 0.1160 - accuracy: 0.9674 - val_loss: 0.1145 - val_accuracy: 0.9690
Epoch 5/10
387/387 - 163s - loss: 0.1152 - accuracy: 0.9679 - val_loss: 0.1219 - val_accuracy: 0.9670
Epoch 6/10
387/387 - 158s - loss: 0.1152 - accuracy: 0.9675 - val_loss: 0.1188 - val_accuracy: 0.9680
Epoch 7/10
387/387 - 158s - loss: 0.1148 - accuracy: 0.9682 - val_loss: 0.1172 - val_accuracy: 0.9685
Epoch 8/10
387/387 - 161s - loss: 0.1145 - accuracy: 0.9681 - val_loss: 0.1142 - val_accuracy: 0.9693
Epoch 9/10
387/387 - 166s - loss: 0.1133 - accuracy: 0.9687 - val_loss: 0.1151 - val_accuracy: 0.9689
Epoch 10/10
387/387 - 158s - loss: 0.1136 - accuracy: 0.9683 - val_loss: 0.1137 - 

In [20]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
387/387 - 161s - loss: 0.1113 - accuracy: 0.9691 - val_loss: 0.1102 - val_accuracy: 0.9704
Epoch 2/10
387/387 - 161s - loss: 0.1108 - accuracy: 0.9691 - val_loss: 0.1102 - val_accuracy: 0.9708
Epoch 3/10
387/387 - 163s - loss: 0.1113 - accuracy: 0.9691 - val_loss: 0.1107 - val_accuracy: 0.9705
Epoch 4/10
387/387 - 164s - loss: 0.1108 - accuracy: 0.9691 - val_loss: 0.1100 - val_accuracy: 0.9706
Epoch 5/10
387/387 - 163s - loss: 0.1111 - accuracy: 0.9693 - val_loss: 0.1103 - val_accuracy: 0.9706
Epoch 6/10
387/387 - 162s - loss: 0.1103 - accuracy: 0.9691 - val_loss: 0.1105 - val_accuracy: 0.9705
Epoch 7/10
387/387 - 163s - loss: 0.1109 - accuracy: 0.9692 - val_loss: 0.1104 - val_accuracy: 0.9706
Epoch 8/10
387/387 - 163s - loss: 0.1098 - accuracy: 0.9697 - val_loss: 0.1101 - val_accuracy: 0.9712
Epoch 9/10
387/387 - 161s - loss: 0.1105 - accuracy: 0.9694 - val_loss: 0.1097 - val_accuracy: 0.9710
Epoch 10/10
387/387 - 163s - loss: 0.1108 - accuracy: 0.9693 - val_loss: 0.1096 - 

In [21]:
history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
387/387 - 152s - loss: 0.1105 - accuracy: 0.9692 - val_loss: 0.1101 - val_accuracy: 0.9710
Epoch 2/10
387/387 - 152s - loss: 0.1097 - accuracy: 0.9694 - val_loss: 0.1099 - val_accuracy: 0.9710
Epoch 3/10
387/387 - 152s - loss: 0.1102 - accuracy: 0.9694 - val_loss: 0.1095 - val_accuracy: 0.9710
Epoch 4/10
387/387 - 152s - loss: 0.1103 - accuracy: 0.9695 - val_loss: 0.1097 - val_accuracy: 0.9710
Epoch 5/10
387/387 - 152s - loss: 0.1094 - accuracy: 0.9694 - val_loss: 0.1091 - val_accuracy: 0.9712
Epoch 6/10
387/387 - 152s - loss: 0.1102 - accuracy: 0.9695 - val_loss: 0.1098 - val_accuracy: 0.9709
Epoch 7/10
387/387 - 152s - loss: 0.1090 - accuracy: 0.9694 - val_loss: 0.1091 - val_accuracy: 0.9714
Epoch 8/10
387/387 - 151s - loss: 0.1097 - accuracy: 0.9695 - val_loss: 0.1089 - val_accuracy: 0.9714
Epoch 9/10
387/387 - 152s - loss: 0.1104 - accuracy: 0.9691 - val_loss: 0.1092 - val_accuracy: 0.9709
Epoch 10/10
387/387 - 152s - loss: 0.1096 - accuracy: 0.9696 - val_loss: 0.1092 - 